# MERFISH Data reorganization

In [15]:
import pandas as pd
import numpy as np
import os, sys, time
import re
import shutil
from importlib import reload
# this version doesn't have ChromAn compiled, so directly load from src:
sys.path.append(r"/lab/weissman_imaging/puzheng/Softwares/ChromAn")
import src

In [4]:
from src.file_io.data_organization import search_fovs_in_folders

data_folder = r"/lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH"

fds, fovs = search_fovs_in_folders(data_folder)


- searching in folder: /lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH
-- 19 folders, 101 fovs detected.


In [5]:
date = os.path.basename(data_folder).split('-')[0]

sample = 'mOP'
library = 'M1'

print(date, sample, library)

20210717 mOP M1


In [6]:
from src.file_io.data_organization import create_folder
# shutter files
ref_image_type = '750_647_561_488_405_s13' # ref 58 / 65
image_type = '750_647_561_s13' # ref 36/39
# fiducial beads
ref_channel = 561
ref_frame = 11 # start from 0!

# target folder
merfish_data_folder = r'/lab/weissman_imaging/puzheng/MERFISH_data'

# target folder:
target_folder = os.path.join(merfish_data_folder, f'{date}-{sample}_{library}')
create_folder(target_folder)


Folder: /lab/weissman_imaging/puzheng/MERFISH_data/20210717-mOP_M1 already exists


('0', '0')

In [16]:
#sel_fovs = []
folder_regexp = r'H([0-9]+)[MRCP]([0-9+])'

ref_round = 0
#fov_ids = np.arange(2)
overwrite = False
remove_source = False

for _fov_id, _fov_name in enumerate(fovs):
    # check if this fov is selected:
    if 'sel_fovs' in locals() and _fov_id not in sel_fovs:
        continue # skip if not selected
    # prceed for the rest:
    for _fd in fds:
        # match round and hyb
        _match = re.search(folder_regexp, os.path.basename(_fd))
        if _match is None:
            continue
        _round, _region = _match.groups()
        # mutliple files for each image:
        _files = [os.path.join(_fd, _fl) for _fl in os.listdir(_fd) if _fov_name.split(os.extsep)[0] in _fl]
        # reference round specified
        if _round == ref_round:
            _image_type = ref_image_type
        else:
            _image_type = image_type
    
        # target file
        _target_files = [os.path.join(target_folder, f"{_image_type}_{_fov_id}_{_round}{os.extsep}{_fl.split(os.extsep)[-1]}") 
                         for _fl in _files]
        for _fl, _tar_fl in zip(_files, _target_files):
            if remove_source:
                print(f"move {_fl} to {_tar_fl}")
                shutil.move(_fl, _tar_fl)
            else:
                if not os.path.isfile(_tar_fl) or overwrite:
                    print(f"copy {_fl} to {_tar_fl}")
                    shutil.copyfile(_fl, _tar_fl)
                else:
                    print(f"{_tar_fl} already exist, skip")

copy /lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH/H0M0/Conv_zscan_000.xml to /lab/weissman_imaging/puzheng/MERFISH_data/20210717-mOP_M1/750_647_561_s13_0_0.xml
copy /lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH/H0M0/Conv_zscan_000.dax to /lab/weissman_imaging/puzheng/MERFISH_data/20210717-mOP_M1/750_647_561_s13_0_0.dax
copy /lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH/H0M0/Conv_zscan_000.inf to /lab/weissman_imaging/puzheng/MERFISH_data/20210717-mOP_M1/750_647_561_s13_0_0.inf
copy /lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH/H0M0/Conv_zscan_000.off to /lab/weissman_imaging/puzheng/MERFISH_data/20210717-mOP_M1/750_647_561_s13_0_0.off
copy /lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH/H0M0/Conv_zscan_000.power to /lab/weissman_imaging/puzheng/MERFISH_data/20210717-mOP_M1/750_647_561_s13_0_0.power
copy /lab/weissman_imaging/puzheng/20210717-P_brain_MO4_MERFISH/H2M2_BIT3-4/Conv_zscan_000.xml to /lab/weissman_imaging/puzhen

## Check file length

In [23]:
print(len(os.listdir(target_folder)), len(fds) * len(fovs) * 5)

9595 9595
